# I decided to add the OpenAI bit here to see if NLP valence measures act differently to a GPT-generaged summary of lyrics than to the lyrics themselves.


In [46]:
##Step 0: Red in data and nltk

df = pd.read_pickle('BeatlesSongsNLP.pkl')

import pandas as pd
import openai
import os

# Set your OpenAI API key as an environment variable
#os.environ["OPENAI_API_KEY"] = "foo"
#os.environ["OPENAI_API_ORG"] = "bar"


# Get your OpenAI API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")
# api_org = os.environ.get("OPENAI_API_ORG")

# Print the API key to verify that it was retrieved correctly
#print(api_key)


In [47]:
openai.api_key = api_key
#openai.organization = api_org



In [48]:
#Get a list of which OpenAI models are available to me through the API

response = openai.Model.list()
model_list = [model['id'] for model in response['data']]
model_list.sort()
model_list


['ada',
 'ada-code-search-code',
 'ada-code-search-text',
 'ada-search-document',
 'ada-search-query',
 'ada-similarity',
 'babbage',
 'babbage-code-search-code',
 'babbage-code-search-text',
 'babbage-search-document',
 'babbage-search-query',
 'babbage-similarity',
 'code-davinci-edit-001',
 'code-search-ada-code-001',
 'code-search-ada-text-001',
 'code-search-babbage-code-001',
 'code-search-babbage-text-001',
 'curie',
 'curie-instruct-beta',
 'curie-search-document',
 'curie-search-query',
 'curie-similarity',
 'davinci',
 'davinci-instruct-beta',
 'davinci-search-document',
 'davinci-search-query',
 'davinci-similarity',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-16k-0613',
 'gpt-4',
 'gpt-4-0314',
 'gpt-4-0613',
 'text-ada-001',
 'text-babbage-001',
 'text-curie-001',
 'text-davinci-001',
 'text-davinci-002',
 'text-davinci-003',
 'text-davinci-edit-001',
 'text-embedding-ada-002',
 'text-search-ada-doc-001',
 'text-sea

In [49]:
## NOTE - this uses the older (soon to be depricated) API method for earlier LLMs. I'm including here for comparison purposes only


def summarize_text_completion(text, model_engine):
    prompt = (f"Please summarize the following song lyrics:\n{text}\n\nSummary:")
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.5,
    )
    summary = response.choices[0].text.strip()
    return summary

# Use the model_engine as the column name
model_engine = "text-davinci-003"
df[model_engine] = df["lyrics"].apply(lambda x: summarize_text_completion(x, model_engine))



In [57]:
print(df["text-davinci-003"])

0      The song reflects on the beauty of the univers...
1      The lyrics of this song tell of a person's amb...
2      The speaker is expressing their unconditional ...
3      The singer is expressing their love and devoti...
4      This song is about a person expressing their l...
                             ...                        
193    The song encourages everyone to come together ...
194    The narrator is warning someone that if they d...
195    The narrator is feeling dejected because their...
196    The lyrics describe a person's journey to find...
197    The song is about reminiscing on the past and ...
Name: text-davinci-003, Length: 198, dtype: object


In [54]:
## Now use the newer API method for GPT-4 that is conversation based
# ChatCompletion

def summarize_ChatCompletion(text, model_engine):
    response = openai.ChatCompletion.create(
      model=model_engine,
      messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following lyrics for me: {text}. Begin each response with a valence score for the lyrics between -1 and 1. Your response should be begin with the valence score, then a comma, then the summary of the lyrics. Example: '0.80, This is a summary of the lyrics.'"},
        ]
    )

    # Return the content of the message from the assistant
    return response['choices'][0]['message']['content']

# Use the model_engine as the column name
model_engine = "gpt-4"
df[model_engine] = df["lyrics"].apply(lambda x: summarize_ChatCompletion(x, model_engine))


In [56]:
print(df["gpt-4"])

0      0.50, The lyrics express a deep meditation and...
1      0.60, The lyrics express the singer's aspirati...
2      0.90, These lyrics express mutual affection an...
3      0.90, The lyrics express longing and commitmen...
4      0.90, These lyrics consist of a playful counti...
                             ...                        
193    0.90, These lyrics are about dancing and singi...
194    0.5, The lyrics are a warning to someone about...
195    -0.80, The lyrics convey a deep sense of sadne...
196    0.85, The lyrics express the narrator's realiz...
197    0.65, These lyrics express a longing for freed...
Name: gpt-4, Length: 198, dtype: object


In [58]:
df.columns

Index(['Id', 'SongKey', 'song', 'songwriters', 'lead_vocals', 'year', 'cover',
       'Era', 'first_release_date', 'max_key', 'mode', 'avg_danceability',
       'avg_energy', 'avg_loudness', 'avg_speechiness', 'avg_acousticness',
       'avg_instrumentalness', 'avg_liveness', 'avg_valence', 'avg_tempo',
       'avg_duration_ms', 'avg_time_signature', 'Song_y', 'lyrics',
       'avg_popularity', 'core_catalogue_releases', 'Canonical_album',
       'first_vocalist', 'Paul_Song', 'John_Song', 'George_Song', 'Ringo_Song',
       'popularity', 'BB_name', 'BB_artist', 'BB_debut_date',
       'BB_peak_position', 'BB_peak_date', 'BB_weeks_on_chart',
       'RollingStonePosition', 'RobSegment', 'LauraSegment', 'EmilySegment',
       'OliviaSegment', 'BrianSegment', 'JackieSegment', 'vader_neg',
       'vader_neu', 'vader_pos', 'vader_compound', 'roberta_neg',
       'roberta_neu', 'roberta_pos', 'Pipeline_sentiment_label',
       'Pipeline_sentiment_score', 'FinetunedSentiment', 'distilbertsent

In [62]:
## I overlaoded the GPT-4 value to include both a sentiment score and the summary. This code splits the column into two columns, one for the sentiment score and one for the summary.


# Split the column into two
df[['GPT-4_valence_score', 'GPT4_summary']] = df['gpt-4'].str.split(', ', n=1, expand=True)

# Convert the 'valence_score' column to numeric
df['GPT-4_valence_score'] = pd.to_numeric(df['GPT-4_valence_score'])



In [69]:
pd.set_option('display.max_rows', 100)

## Show me these columns in the dataframe: song, lyrics, text_davinci_003, gpt_4 
df_LLM = df[["song", "lyrics", "text-davinci-003", "GPT4_summary", "GPT-4_valence_score"]]
df_LLM

,song,lyrics,text-davinci-003,GPT4_summary,GPT-4_valence_score
0,Across the Universe,"Words are flowing out like endless rain into a paper cup, They slither while they pass, they slip away across the universe Pools of sorrow, waves of joy are drifting through my open mind, Possessing and caressing me. Jai guru de va om Nothing's gonna change my world, Nothing's gonna change my world. Images of broken light which dance before me like a million eyes, That call me on and on across the universe, Thoughts meander like a restless wind inside a letter box they Tumble blindly as they make their way Across the universe Jai guru de va om Nothing's gonna change my world, Nothing's gonna change my world. Sounds of laughter shades of earth are ringing Through my open views inviting and inciting me Limitless undying love which shines around me like a Million suns, it calls me on and on Across the universe Jai guru de va om Nothing's gonna change my world, Nothing's gonna change my world.","The song reflects on the beauty of the universe and the feeling of limitless love that it conveys. It speaks of the power of words and how they can slip away, and of the emotions of sorrow and joy that come to mind. The chorus is a reminder that no matter what, nothing will change the world.","The lyrics express a deep meditation and introspection from the perspective of a calmed, untouchable mind. Powerful images like words flowing like endless rain, pools of sorrow, waves of joy, and broken light dancing suggest a contemplation of both the internal world of emotions and the external world of experience. Despite all the sensory claims and strong emotions sparked by these experiences, the singer insists that nothing can change their internal world. The recurring phrase ""Jai guru de va om"" is a mantra referring to a spiritual guide or enlightenment, further highlighting the theme of calm mental resolve amidst the chaotic flow of life's experiences.",0.50
1,Act Naturally,"They're gonna put me in the movies They're gonna make a big star out of me We'll make a film about a man that's sad and lonely And all I gotta so is act naturally Well, I'll bet you I'm gonna be a big star Might win an Oscar you can never tell The movies gonna make me a big star 'Cause I can plat the part so well Well I hope you come and see me in the movies Then I know that you will plainly see The biggest fool that ever hit the big time And all I gotta do is act naturally We'll make the scene about a man that's sad and lonely And beggin down upon his bended knee I'll play the part but I won't need rehearsal All I gotta do is act naturally Well, I'll bet you I'm gonna be a big star Might win an Oscar you can never tell The movies gonna make me a big star 'Cause I can plat the part so well Well I hope you come and see me in the movies Then I know that you will plainly see The biggest fool that ever hit the big time And all I gotta do is act naturally","The lyrics of this song tell of a person's ambition to become a movie star. They express confidence in their ability to act naturally and play the part well, and believe they could even win an Oscar. The song also expresses hope that the listener will come and see them in the movies, where they will be seen as the ""biggest fool that ever hit the big time"" but still be able to act naturally.","The lyrics express the singer's aspirations about becoming a big movie star by simply acting naturally. The singer believes they could potentially win an Oscar due to their acting skills. The role they play is of a sad and lonely man, a part that they feel they can authentically portray without needing any rehearsal. They hope the audience will recognize them in this role and see how they became successful by just being themselves. They also suggest that they might end up being perceived as the biggest fool to have ever become famous.",0.60
2,All I've Got to Do,"Whenever I want you around, yeah. All I gotta do, Is call you on the phone, And you'll c

In [64]:
pd.to_pickle(df, 'BeatlesSongsNLPwithGPT.pkl')

In [70]:
df.to_csv('BeatlesSongsNLPwithGPT.csv', index=False) # dump processed file